In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import collections

from scipy import stats
from datetime import datetime
from tqdm import tqdm
import itertools
import networkx as nx
import community
import seaborn as sns
import operator

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_av = pd.read_csv('../input/avocado-prices-2020/avocado-updated-2020.csv')
df_av.head()

## Exploratory Data Analysis
### Price Distribution Examination

To get us started, let us take a look at the distribution of avocado prices. We must consider three things:
1. There are two kinds of avocadoes `Conventional` and `Organic`.
2. The data contains values that are aggregates of several regions.
3. The data contains information on both time and space.

To address these issues we will do the following:
1. Filter the data for the aggregate `region` figures.
2. Disaggregate `Conventional` and `Organic` avocadoes.
3. Begin with a total price view then drill down to consider `region` and `Date` differences.

### Filter the data for the aggregated region figures
Looking at the list below we can see several regions that we should omit. Apart from the specific aggregate regions, we shall also only choose the columns that are of interest to us.

In [ ]:
print(sorted(list(set(df_av['geography'].values))))

### Data Wrangling

In [ ]:
delete_list = ['Total U.S.','West','Southeast','South Central',
               'Plains','California','South Carolina',
               'Northeast','Great Lakes','Midsouth','West Tex/New Mexico']
df_av = df_av[['date','average_price','type','geography','year']]
df_av = df_av[~df_av['geography'].isin(delete_list)]
df_av.head()

### Price Examination of Conventional and Organic Avocadoes
We can see that `Conventional` avocadoes are generally less expensive than `Organic` avocadoes. Likewise, the `Conventional` avocadoes are far more concentrated around their mean. This is highlighted by the differences in their **Means**, **Medians**, **Skewness**, and level of **Standard Deviation** as seen in the below table.

This could highlight that `Conventional` avocadoes are subject to greater market competition, compelling sellers to keep their prices close to market prices. This greater adherence to market conditions can be seen quantitatively through its lower **Standard Deviation**. `Organic` avocadoes by contrast are both more expensive and have a much more diverse range of values. This can be attributed to the greater product differentiation `Organic` avocadoes have. Additionally, the buyers of `Organic` avocadoes may also be willing to accept a greater variety of prices depending on the conditions the avocadoe was grown in.

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
fig.suptitle('Price Distribution of Avocadoes', fontsize = 22)
ax1 = sns.distplot(df_av[df_av['type']=='conventional']["average_price"], 
                   color = 'b', 
                   label = 'Conventional')
ax2 = sns.distplot(df_av[df_av['type']=='organic']["average_price"], 
                   color = 'g', 
                   label = 'Organic');
plt.legend(title = 'Avocado Type', title_fontsize = 12);

In [ ]:
df_conv_pr = df_av[df_av['type']=='conventional']["average_price"]
df_orgn_pr = df_av[df_av['type']=='organic']["average_price"]

pd.DataFrame([['Mean', df_conv_pr.mean(),df_orgn_pr.mean()],
              ['Median', df_conv_pr.median(),df_orgn_pr.median()],
              ['Std', df_conv_pr.std(),df_orgn_pr.std()],
              ['Skewness', df_conv_pr.skew(),df_orgn_pr.skew()]],
              columns = ['Metric','Conventional','Organic'])

### Disaggregating the Data using Factor Plots
We also have to consider the fact that the prices affected by both the `region` and the `Date` features. To give us a better idea of the differences we can make use of Factor Plots.

Immediately we can see the vast difference between regions with a noticable difference between regions in the southern United States and their northern counterparts. Intuitively, this can be attributed to the greater cost of moving avocadoes from the south to the north. For those who are unfamiliar, **avocadoes** are primarily produced in [**California**](https://www.agmrc.org/commodities-products/fruits/avocados#:~:text=According%20to%20NASS%2C%20California%20now,followed%20by%20Florida%20and%20Hawaii.) with other varieties being sources from [**Latin America**](https://en.wikipedia.org/wiki/List_of_countries_by_avocado_production). Both centers of production lead to lower prices in the Southerwestern area of the United States.

The relationship with `Date` is less certain. However, there was a notable jump in the **2017** prices across all `regions` and affecting both `types` of avocadoes. One reason for the spike was a [notable drought in **California**](https://www.bbc.com/news/business-39768480), avocades of course being a particulary drough-sensitive crop. Furthermore, we can see that avocado prices have hit record lows in **2020**.

In [ ]:
factor_c = sns.factorplot('average_price','geography',hue='year',
                height = 12,aspect=0.7, 
                palette='rocket', 
                join = False,
                data = df_av[df_av['type']=='conventional'],
                );

In [ ]:
factor_o = sns.factorplot('average_price','geography',hue='year',
                height = 12,aspect=0.7, 
                palette='GnBu', 
                join = False,
                data = df_av[df_av['type']=='organic'],
                );

### Checking for Normality
To enable a better understanding of the underlying data, we can check for the level of normality of the data. In this case, we check if price distribution can be considered **Normally Distributed**. To do this we make use of the **Kolmogorv-Smirnov** test.

The hypothesis of the test is:<br>

**H<sub>0</sub>** : The Sample Distributions are from the same continuous distribution.

**H<sub>1</sub>**  : The Sample Distributions are **NOT** from the same continuous distribution.

By setting the **Kolmogorv-Smirnov** to `norm`, we are essentially checking whether or not the prices per `region` are normally distributed. In our case, all the prices per region are **NOT Normally Distributed**.

In [ ]:
def norm_checker(df, year_thresh = 2015):
    df = df[df['year']>=year_thresh]
    final_data = []
    for r in tqdm(sorted(set(df['geography']))):
        prices_c = df.loc[(df['geography'] == r) & (df['type'] == 'conventional')]['average_price']
        prices_o = df.loc[(df['geography'] == r) & (df['type'] == 'organic')]['average_price']

        ks_statistic_c, p_value_c = stats.kstest(prices_c, 'norm')
        ks_statistic_o, p_value_o = stats.kstest(prices_o, 'norm')
        
        if p_value_c > 0.05:
            normal_c = 'normal'
        else:
            normal_c = 'not normal'
            
        if p_value_o > 0.05:
            normal_o = 'normal'
        else:
            normal_o = 'not normal' 
        
        data = [r,ks_statistic_c,p_value_c,ks_statistic_o,p_value_o, normal_c,normal_o]
        final_data.append(data)
    return pd.DataFrame(final_data, columns = ['Region', 'Kstat_conv','p_conv','Kstat_org','p_org','result_c','result_o'])
norms  = norm_checker(df_av, 2016)
norms.head()

In [ ]:
def corr_creator_spearman(df, year_thresh = 2015):
    df = df[df['year']>=year_thresh]
    all_regions = sorted(set(df['geography']))
    region_combos = list(itertools.combinations(all_regions, 2))
    final_data_c = []
    final_data_o = []
    for c in tqdm(region_combos):
        
        df.loc[(df['geography'] == c[0]) & (df['type'] == 'organic')][['date','average_price']]
        df.loc[(df['geography'] == c[1]) & (df['type'] == 'organic')][['date','average_price']]
        merged_c = pd.merge(df.loc[(df['geography'] == c[0]) & (df['type'] == 'conventional')][['date','average_price']],
                               df.loc[(df['geography'] == c[1]) & (df['type'] == 'conventional')][['date','average_price']],
                               on='date')
        
        merged_o = pd.merge(df.loc[(df['geography'] == c[0]) & (df['type'] == 'organic')][['date','average_price']],
                            df.loc[(df['geography'] == c[1]) & (df['type'] == 'organic')][['date','average_price']],
                            on='date')

        spearmanr_c = stats.spearmanr(merged_c['average_price_x'],
                                      merged_c['average_price_y'])
        
        spearmanr_o = stats.spearmanr(merged_o['average_price_x'],
                                      merged_o['average_price_y'])
        

        c_data = [c[0],c[1], round(spearmanr_c[0],6),round(spearmanr_c[1],6)]
        o_data = [c[0],c[1], round(spearmanr_o[0],6),round(spearmanr_c[1],6) ]
        
        final_data_c.append(c_data)
        final_data_o.append(o_data)
        
    df_c = pd.DataFrame(final_data_c, columns = ['region_1','region_2','spearman_corr','significance'])
    df_o = pd.DataFrame(final_data_o, columns = ['region_1','region_2','spearman_corr','significance'])
    return [df_c,df_o]

df_spear = corr_creator_spearman(df_av, 2016)
df_spear

### Choosing Threshold Metric
The next task is the selection of correlation that we should use to create linkages. Admittedly, the methodology for this is more analytical than theoretical. In choosing the appropriate threshold we have to consider the following:
1. Networks where nodes exhibiit **low correlation** may not be useful to analysts.
2. Networks where there is a **highly atomized** are also not useful.
3. Large networks are **computationally expensive** to run tests and simulations on.

The below tables show us that the behaviour of the `Conventional` avocado network seems very different from the `Organic` network. This highlight that the difference between the two markets runs deeper than the cost of production and consumer's willingness to pay. On a larger scale, the correlations between regional markets are quite different. Practically speaking, it would be wise to choose the **highest correlation** that still yields a large **Giant Component (the largest portion of the network)**.

For the `Conventional` market it seems that this figure is **0.70** while for the `Organic` market it is **0.50**. We shall continue the analysis using those specific threshold figures.

In [ ]:
def construct_network_spear(df, signthresh = 0.05, corrthresh = 0.50):
    av_G = nx.Graph()
    for i in tqdm(df.iterrows()):
        if i[1][3] <= signthresh:
            av_G.add_edge(i[1][0], i[1][1], weight = i[1][2])
        else:
            av_G.add_node(i[1][0])
            av_G.add_node(i[1][1])
    
    av_G.remove_edges_from(nx.selfloop_edges(av_G))
    low_weight_edges = [i[:2]  for i in av_G.edges(data=True) if abs(i[2]['weight']) <= corrthresh]
    av_G.remove_edges_from(low_weight_edges)
            
    return av_G

def get_network_metrics(df):
    
    thresh_list = np.arange(0.1,1,0.05)
    list_of_graphs = [construct_network_spear(df, 0.05, thresh) for thresh in thresh_list]

    final_list = []
    for n, g in enumerate(list_of_graphs):
        GC_nodes = max(nx.connected_components(g), key=len)
        GC = g.subgraph(GC_nodes).copy()
        SC = [g.subgraph(i).copy() for i in 
              list(nx.connected_components(g)) if i != GC_nodes]
        L = g.size() 
        avg_deg = float(L) / g.order()  
        
        #Try/Except logic as the function will error if SC == 0
        try:
            SC_average_nodes = np.mean([len(i) for i in SC])
            SC_max_nodes = np.max([len(i) for i in SC])
            SC_min_nodes = np.min([len(i) for i in SC])
        except:
            SC_average_nodes = np.nan
            SC_max_nodes = np.nan
            SC_min_nodes = np.nan
        
        data_point = (L, avg_deg, len(GC), len(SC), 
                      SC_average_nodes, SC_max_nodes, SC_min_nodes, thresh_list[n])
        final_list.append(data_point)
    return pd.DataFrame(final_list,
                        columns =  ['Size','Ave Degree',
                                    '# of Nodes in GC','# of SC', 'Mean of SCs',
                                    'Max of SCs', 'Min of SCs', 'Corr Thresh'])

df_spear_c_metrics = get_network_metrics(df_spear[0])
df_spear_o_metrics = get_network_metrics(df_spear[1])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].set_title('Conventional', fontsize = 17)
ax[0].plot(df_spear_c_metrics['Corr Thresh'],
           df_spear_c_metrics['# of Nodes in GC'],
           color = 'blue', linewidth = 1.75)
ax[0].axvline(x=0.70, color='r', linestyle='--')

ax[1].set_title('Organic', fontsize = 17)
ax[1].plot(df_spear_o_metrics['Corr Thresh'],
           df_spear_o_metrics['# of Nodes in GC'],
           color = 'green', linewidth = 1.75)
ax[1].axvline(x=0.50, color='r', linestyle='--')
fig.tight_layout();

In [ ]:
df_spear_c_metrics

In [ ]:
df_spear_o_metrics

In [ ]:
### Create the Spearman Correlation DataFrames
spear_c_network = construct_network_spear(df_spear[0], 0.05, 0.70)
spear_o_network = construct_network_spear(df_spear[1], 0.05, 0.50)

def plot_single_network(network, title):
    plt.figure(figsize=(15,7)) 
    
    g_corr = network.subgraph(sorted(nx.connected_components(network), key = len, reverse = True)[0])
    d = dict(network.degree)
    d_g = dict(g_corr.degree)
    plt.suptitle(title, fontsize=20, y=0.95)
    pos = nx.spring_layout(network, seed = 45);
    nx.draw_networkx_nodes(network , pos, cmap=plt.cm.RdYlBu_r,
                           node_color = [val for (node, val) in network.degree()],
                           node_size=[v * 25 for v in d.values()]);
    nx.draw_networkx_edges(network , pos, width=0.75, alpha=0.45, 
                            edge_color="grey");
    nx.draw_networkx_labels(network, pos, font_color  = 'grey')
    plt.show();
    
    plt.figure(figsize=(15,7)) 
    plt.suptitle(f'{title} Giant Component', fontsize=20, y=0.95)
    pos = nx.spring_layout(g_corr, seed = 45);
    nx.draw_networkx_nodes(g_corr , pos, cmap=plt.cm.RdYlBu_r,
                           node_color = [val for (node, val) in g_corr.degree()],
                           node_size=[v * 25 for v in d_g.values()]);
    nx.draw_networkx_edges(g_corr , pos, width=0.75, alpha=0.45, 
                            edge_color="grey");
    nx.draw_networkx_labels(g_corr, pos, font_color  = 'grey')
    
    plt.show();
    
def degree_dist(network1, network2):
    degrees1 = [d for n,d in network1.degree()]
    degrees2 = [d for n,d in network2.degree()]
    
    fig, ax = plt.subplots(figsize=(15,7))
    fig.suptitle('Degree Distribution of Networks', fontsize = 22)
    ax1 = sns.histplot(degrees1, edgecolor='white', linewidth=0.75,
                       color = 'b', kde = False, binwidth=1,alpha = 0.55,
                       label = 'Conventional')
    ax2 = sns.histplot(degrees2, edgecolor='white', linewidth=0.75,
                       color = 'g', kde = False, binwidth=1,alpha = 0.55,
                       label = 'Organic');
    plt.legend(title = 'Degree Distribution', title_fontsize = 12);

### Network Analysis
Examining the networks, we can see the vast difference between the `Conventional` and `Organic` markets. The `Conventional` markets being far more closely linked, evidenced by their high degree of connectivity despite their higher correlation threshold. `Organic` markets are far more diverse, almost splitting apart at even their lower threshold. 

Looking into the degree distribution, we find that the two networks have extremely different underlying structures. In the `Conventional` market we see that the central nodes dominate when it comes to degree of connections. The `Organic` market on the other hand has a much more distributed level of degrees. This implies that prices tend to be stickier in the `Conventional` and far more reactive in the `Organic`. 

In [ ]:
def plot_single_network(network, title):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    g_corr = network.subgraph(sorted(nx.connected_components(network), key = len, reverse = True)[0])
    d = dict(network.degree)
    d_g = dict(g_corr.degree)
    ax[0].set_title(title, fontsize=20, y=1.0)
    pos = nx.spring_layout(network, seed = 45)
    nx.draw_networkx_nodes(network , pos, cmap=plt.cm.RdYlBu_r,
                           node_color = [val for (node, val) in network.degree()],
                           node_size=[v * 25 for v in d.values()], 
                           ax = ax[0]);
    nx.draw_networkx_edges(network , pos, width=0.75, alpha=0.45, 
                            edge_color="grey", ax = ax[0]);
    nx.draw_networkx_labels(network, pos, font_color  = 'grey', 
                            ax = ax[0])
    
    ax[1].set_title(f'{title} Giant Component', fontsize=20, y=1.0)
    pos = nx.spring_layout(g_corr, seed = 45);
    nx.draw_networkx_nodes(g_corr , pos, cmap=plt.cm.RdYlBu_r,
                           node_color = [val for (node, val) in g_corr.degree()],
                           node_size=[v * 25 for v in d_g.values()],
                           ax = ax[1]);
    nx.draw_networkx_edges(g_corr , pos, width=0.75, alpha=0.45, 
                            edge_color="grey",ax = ax[1]);
    nx.draw_networkx_labels(g_corr, pos, font_color  = 'grey',
                            ax = ax[1])
    
    fig.tight_layout();

In [ ]:
plot_single_network(spear_c_network, 'Conventional Avocado Price Network')

In [ ]:
plot_single_network(spear_o_network, 'Organic Avocado Price Network')

In [ ]:
degree_dist(spear_c_network,spear_o_network)

In [ ]:
### Get top 5 nodes with the most degrees
deg_c = sorted([(n,d) for n,d in spear_c_network.degree()], key = lambda x:(-x[1], x[0]))[:5]
deg_o = sorted([(n,d) for n,d in spear_o_network.degree()], key = lambda x:(-x[1], x[0]))[:5]

### Plot the distribution
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].set_title('Conventional', fontsize = 17)
ax[0].bar(range(len(deg_c)), 
          [val[1] for val in deg_c],
          color = 'blue', 
          alpha = 0.50,
          align='center')
ax[0].set_xticks(range(len(deg_c)))
ax[0].set_xticklabels([val[0] for val in deg_c])
ax[0].tick_params(rotation=45)

ax[1].set_title('Organic', fontsize = 17)
ax[1].bar(range(len(deg_o)), 
          [val[1] for val in deg_o],
          color = 'green', 
          alpha = 0.50,
          align='center')
ax[1].set_xticks(range(len(deg_o)))
ax[1].set_xticklabels([val[0] for val in deg_o])
ax[1].tick_params(rotation=45)

fig.tight_layout()

### Measuring Interconnectivity

The interconnectivity of the graph helps us improve our understanding of the its dynamics. To a lesser extent we get an idea through the **Average Degree** and **Nodes in the Giant Component** metrics, however such measures are quite rough and may not always give the full picutre. Below are two measurements we shall focus on:<br>

**Clustering Coefficient** - The level of interconnectivity a system has based on the level of connectiviy of each node.<br>
**Rich Club Coefficient** - The level of interconnectivity of the nodes with the most connections with each other. <br>

For the **Clustering Coefficient**, we can see that the `Conventional` market has a notable decline after the most highly connected nodes. The `Organic` market takes much longer to drop off. This highlights the difference in connectivity level hinted at when visually examining the graph. 

The **Rich Club Coefficient** tells quite an interesting story. The `Conventional` market shows a a coefficient that consistently grows and then shoots up and plateaus at degrees greater than 10. The `Organic` market shows a much slower increase over the number of degrees; however, it exhibits a massive jump also at degrees greater than 10. This highlights the centralized nature of the `Conventional` market, with a regions connectivity with other regions growing consistently as it gains more connections. The decentralized nature of the `Organic` market means that even though a region increases its connectivity with other nodes will not guarantee it suddenly "joins the club". It seems that it must connect with a specific hub to gain the benefits.

In [ ]:
### Get the Clustering Coefficiencts of the Graph
cc_c = sorted(nx.clustering(spear_c_network).items(), key = lambda x:(-x[1],x[0]))
cc_o = sorted(nx.clustering(spear_o_network).items(), key = lambda x:(-x[1],x[0]))

### Plot the distribution
fig, ax = plt.subplots(1,2,figsize=(15,5))
fig.suptitle('Clustering Coefficients', fontsize=22)
ax[0].set_title('Conventional', fontsize = 17)
ax[0].bar(range(len(cc_c)), 
          [val[1] for val in cc_c],
          color = 'blue', 
          alpha = 0.50,
          align='center')

ax[1].set_title('Organic', fontsize = 17)
ax[1].bar(range(len(cc_o)), 
          [val[1] for val in cc_o],
          color = 'green', 
          alpha = 0.50,
          align='center')

fig.tight_layout();

In [ ]:
### Get the Rich Club Coefficiencts of the Graph
rc_c = nx.rich_club_coefficient(spear_c_network, normalized=False, seed=42)
rc_o = nx.rich_club_coefficient(spear_o_network, normalized=False, seed=42)

### Plot the distribution
fig, ax = plt.subplots(1,2,figsize=(15,5))
fig.suptitle('Rich Club Coefficients', fontsize=22)
ax[0].set_title('Conventional', fontsize = 17)
ax[0].bar(rc_c.keys(),
          rc_c.values(),
          color = 'blue', 
          alpha = 0.50,
          align='center')

ax[1].set_title('Organic', fontsize = 17)
ax[1].bar(rc_o.keys(), 
          rc_o.values(),
          color = 'green', 
          alpha = 0.50,
          align='center')

fig.tight_layout();

### Centrality Measurements
To better understand the structure of the network, we can look into the four primary centrality measurements. <br>
1. **Degree Centrality** - based on **number of direct connections** a node has.
2. **Closeness Centrality** - based on **how short the shortest paths** a node has.
3. **Betweenness Centrality** - based on **how many shortest paths pass** through node.
4. **Eigenvector Centrality** -  based on **connectedness of a node's direct neighbors**.

We can see that for the `Conventional` market, there is a high overlap in their Degree, Closeness, and Eigenvector centralities. Nodes located at the center seemingly having highest centrality values while those at the periphery have much lower values. Betweeness though is concentrated in only a few nodes, studying these nodes should then be of particular importance to market researchers. 

The `Organic` market on the other hand has noticeable overlap in its Degree and Closeness. It seems that each small group of nodes have their own highly connected network. We notice the relative unimportance of the northeastern regions, such as **Albany**, **Northern New England**, and **Boston**. This highlights that the avocado price behaviour in these regions should not be compared to avocado prices in the rest of the United States.Interestingly, we can see that the most in-between nodes sit on the opposite ends of the network. This implies that it can pay off to spend more resources understanding those two particular regions. They may be the key in understanding the behaviour of different markets. 

In [ ]:
def centrality_plotter(network, title):
    g_corr = network.subgraph(sorted(nx.connected_components(network), key = len, reverse = True)[0])
    fig, ax = plt.subplots(2,2, figsize=(20,15))
    
    centralities = [list(nx.degree_centrality(g_corr).values()), 
                    list(nx.closeness_centrality(g_corr).values()), 
                    list(nx.betweenness_centrality(g_corr).values()),
                    list(nx.eigenvector_centrality(g_corr).values())]
    titles = ['Degree Centrality', 'Closeness Centrality', 
              'Betweenness Centrality', 'Eigenvector Centrality']
    fig.suptitle(title, fontsize=25)
    for n, ax in enumerate(ax.flatten()):
        ax.set_title(titles[n], fontsize = 19);
        pos = nx.spring_layout(g_corr, seed = 45)
        
        nc = nx.draw_networkx_nodes(g_corr, pos, node_size=300, cmap=plt.cm.RdYlBu_r,
                                    node_color=centralities[n], ax = ax);
        
        
        nx.draw_networkx_edges(g_corr, pos, alpha=0.40, ax = ax);
        nx.draw_networkx_labels(g_corr, pos, font_color='grey', ax = ax);
        ax.set_axis_off()
        plt.colorbar(nc, ax = ax);

    fig.tight_layout()
    fig.subplots_adjust(top=0.90);

In [ ]:
centrality_plotter(spear_c_network, 'Conventional Avocado Network')

In [ ]:
centrality_plotter(spear_o_network,'Organic Avocado Network')

### Community Detection
Finally, one of the most useful functions of a network is the construction of communities. In this case we make use of the popular **Louvain Method** which works by optimizing the modularity of the network. 

The below graphs show that the algorithm identifies 4 communities for the `Conventional` market and 5 communities for the `Organic` market. However, the exculsion of **Boston** and **New England** in the `Organic` market seems unnecessary. When selecting a framework to analyze the data it could be beneficial to include them as part of the light blue community. 

In [ ]:
def community_plotter(network1, network2):
    titles = ['Conventional Communities','Organic Communities']
    g_corr_1 = network1.subgraph(sorted(nx.connected_components(network1), key = len, reverse = True)[0])
    g_corr_2 = network2.subgraph(sorted(nx.connected_components(network2), key = len, reverse = True)[0])
    
    partition_1 = community.best_partition(g_corr_1, random_state = 42)
    partition_2 = community.best_partition(g_corr_2, random_state = 42)
    
    identifier = [list(partition_1.values()),
                  list(partition_2.values())]
    
    graphs = [g_corr_1,g_corr_2]
    
    fig, ax = plt.subplots(1,2, figsize=(15,7))
    for n, ax in enumerate(ax.flatten()):
        ax.set_title(titles[n], fontsize = 17);
        pos = nx.spring_layout(graphs[n], seed = 45)
        
        nc = nx.draw_networkx_nodes(graphs[n], pos, node_size=300, cmap=plt.cm.RdYlBu_r,
                                    node_color=identifier[n], ax = ax);
        
        nx.draw_networkx_edges(graphs[n], pos, alpha=0.40, ax = ax);
        nx.draw_networkx_labels(graphs[n], pos, font_color='grey', font_size = 9.5, ax = ax);
        
    fig.tight_layout();

In [ ]:
community_plotter(spear_c_network, spear_o_network)

In [ ]:
### Extract Giant Components
g_corr_c = spear_c_network.subgraph(sorted(nx.connected_components(spear_c_network), key = len, reverse = True)[0])
g_corr_o = spear_o_network.subgraph(sorted(nx.connected_components(spear_o_network), key = len, reverse = True)[0])

partition_c = community.best_partition(g_corr_c, random_state = 42)
partition_o = community.best_partition(g_corr_o, random_state = 42)

fig, ax = plt.subplots(1,2, figsize=(15,5))
sns.countplot(x = 0, data = pd.DataFrame(partition_c, index = [0]).T.reset_index(), 
              ax = ax[0], palette =['#000fff', '#00D1FF','#Ffa700', '#Ff1f00'] ).set_title("Count of Conventional Community Memebers", fontsize = 15);
sns.countplot(x = 0, data = pd.DataFrame(partition_o, index = [0]).T.reset_index(), 
              ax = ax[1], palette =['#000fff', '#00D1FF','#f2ff00', '#ff8400','#Ff2900']).set_title("Count of Organic Community Memebers", fontsize = 15);
ax[0].set(xlabel=None)
ax[1].set(xlabel=None)

fig.tight_layout();